Let's grab our data and take a look at it. (I did some basic cleaning and removed irrelevant rows in Google Sheets prior to importing, but that isn't strictly necessary for any of this).

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv('skill-ranks.csv')
df.head(2)

,keyword,rank_mar_10,url,tags,cpc,volume
0,python for loop,22,https://www.dataquest.io/blog/python-for-loop-...,python|q3top10|skill-python-basics|tier3,3.10,90500.0
1,python dictionary,56,https://www.dataquest.io/blog/python-dictionar...,python|skill-python-basics|tier3,0.59,60500.0


The way SEMRush handles tags in exports is kind of annoying, but we can live with it. We've got a bunch of different skill paths, and we could use str.contains to separate each of them into a separate, smaller dataframe, like so:

In [11]:
python_stats = df[df['tags'].str.contains('skill-python-stats')]

python_stats.head(2)

,keyword,rank_mar_10,url,tags,cpc,volume
59,statistics with python,28,https://www.dataquest.io/blog/basic-statistics...,python|skill-python-stats|tier3,1.96,1600.0
64,statistics course,-,NaN,intent_keywords|skill-python-stats|skill-r-sta...,2.72,1000.0


But, it may be easier in the future if we keep this all in one database, so instead, let's set a list of conditions and corresponding values, and then create a new column that'll list _only_ the skillpath tag.

In [12]:
conditions = [
    (df['tags'].str.contains('skill-python-stats') == True),
    (df['tags'].str.contains('skill-r-stats') == True),
    (df['tags'].str.contains('skill-python-ml-intermediate') == True),
    (df['tags'].str.contains('skill-python-ml-intro') == True),
    (df['tags'].str.contains('skills-sql') == True),
    (df['tags'].str.contains('skill-r-basics') == True),
    (df['tags'].str.contains('skill-python-basics') == True),
    (df['tags'].str.contains('skill-python-da-dv') == True),
    (df['tags'].str.contains('skill-python-apis-scraping') == True),
    (df['tags'].str.contains('skill-r-apis-scraping') == True),
    (df['tags'].str.contains('skill-r-dv') == True),
    (df['tags'].str.contains('career-builder') == True),
    ]

values = ['skill-python-stats', 'skill-r-stats', 'skill-python-ml-intermediate', 'skill-python-ml-intro',
         'skills-sql', 'skill-r-basics', 'skill-python-basics', 'skill-python-da-dv', 'skill-python-apis-scraping',
          'skill-r-apis-scraping', 'skill-r-dv', 'career-builder'
         ]

df['skillpath'] = np.select(conditions, values)

df.sample(2)

,keyword,rank_mar_10,url,tags,cpc,volume,skillpath
114,learn data skills,3,https://www.dataquest.io/blog/data-analyst-ski...,career-builder|tier2,NaN,10.0,career-builder
100,kaggle tutorials,19,https://www.dataquest.io/blog/kaggle-tutorial/,skill-python-ml-intermediate,1.99,170.0,skill-python-ml-intermediate


Beautiful! Now, let's set conversion percentages for the content in each of these skillpaths, and add those as a new column we can use as a multiplier. To make it easy for anyone to work with, we'll add easy to change values for all the conversion rates.

Note: these conversion numbers are made-up nonsense for now. We can substitute in the real numbers later!

In [14]:
conditions = [
    (df['tags'].str.contains('skill-python-stats') == True),
    (df['tags'].str.contains('skill-r-stats') == True),
    (df['tags'].str.contains('skill-python-ml-intermediate') == True),
    (df['tags'].str.contains('skill-python-ml-intro') == True),
    (df['tags'].str.contains('skills-sql') == True),
    (df['tags'].str.contains('skill-r-basics') == True),
    (df['tags'].str.contains('skill-python-basics') == True),
    (df['tags'].str.contains('skill-python-da-dv') == True),
    (df['tags'].str.contains('skill-python-apis-scraping') == True),
    (df['tags'].str.contains('skill-r-apis-scraping') == True),
    (df['tags'].str.contains('skill-r-dv') == True),
    (df['tags'].str.contains('career-builder') == True),
    ]

python_stats_conv = 0.01
r_stats_conv = 0.02
python_ml_intermediate_conv = 0.01
python_ml_intro_conv = 0.02
sql_conv = 0.006
r_basics_conv = 0.01
python_basics_conv = 0.01
python_da_dv_conv = 0.02
python_apis_scraping_conv = 0.01
r_apis_scraping_conv = 0.02
r_dv_conv = 0.005
career_builder_conv = 0.01

values = [python_stats_conv, r_stats_conv, python_ml_intermediate_conv, python_ml_intro_conv,
         sql_conv, r_basics_conv, python_basics_conv, python_da_dv_conv, python_apis_scraping_conv,
          r_apis_scraping_conv, r_dv_conv, career_builder_conv
         ]

df['signup_conv'] = np.select(conditions, values)
df.sample(2)

,keyword,rank_mar_10,url,tags,cpc,volume,skillpath,signup_conv
98,kaggle courses,8,https://www.dataquest.io/course/kaggle-fundame...,skill-python-ml-intermediate,3.30,210.0,skill-python-ml-intermediate,0.01
107,learning r programming for beginners,8,https://www.dataquest.io/blog/learn-r-for-data...,r|skill-r-basics|tier3,3.47,110.0,skill-r-basics,0.01


Now we're set up to do some more advanced stuff, but for now, let's take a quick look at the potential conversions (not including long-tail and variations) for the SQL skill path, assuming we captured all traffic (which we obviously cannot, just as a proof of concept).

Below is a hacky (but it works!) way to get at this:

In [18]:
df[(df['skillpath'] == 'skills-sql')]['volume'].sum()*df[(df['skillpath'] == 'skills-sql')]['signup_conv'].mean()

1046.7000000000003

That'll do for now. Next steps:
* Create a column for current estimated traffic based on current rank and keyword volume
* Create a column for if we ranked 2nd for all keywords